程式參考並修改自:
> https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  

解釋程式如何改進之參考文章:
> https://david-exiga.medium.com/music-generation-using-lstm-neural-networks-44f6780a4c5  

額外的中文程式解釋:
> https://github.com/xitu/gold-miner/blob/master/TODO1/how-to-generate-music-using-a-lstm-neural-network-in-keras.md

In [ ]:
# 安裝 music21  # music21 介紹: https://juejin.cn/post/7063827463058489352
! pip install music21

In [1]:
# 讀取檔案用
import glob
# array processing
import numpy
from matplotlib import pyplot

# keras for building deep learning model
import keras
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# 使用music21來進行midi檔案的操作
from music21 import converter, instrument, note, chord, stream, duration
# music21 介紹: https://juejin.cn/post/7063827463058489352


### 從目錄下的 midi 文件中獲取所有的音符和和弦

In [2]:
## """ 從目錄下的 midi 文件中獲取所有的音符和和弦 """

notes = []
rests = []
notes_without_restes = []
rest_to_duration = {}


# 使用glob讀取midi檔案，路徑代號為:
# "./"               -使用當前執行程式所在的資料夾
# "midi_songs/"      -名為midi_songs資料夾
# "*.mid"            -所有結尾為的.mid檔案
# note是音符，將midi檔案裡的音符讀進這個list

for file in glob.glob("./midi_songs/*.mid"): # 讀取目錄檔案路徑中所有midi檔案
    
    # 使用 music21 解析midi文件
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: #如果有樂器部分，取第一個樂器
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse() 
    except: #如果沒有樂器部分，直接取note
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        # 如果是 Note 型別，取音調
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
            notes_without_restes.append(str(element.pitch))
        # 如果是 Rest 型別，取休止符名稱
        if isinstance(element, note.Rest):
            notes.append(str(element.fullName))
            rests.append(str(element.fullName))
            rest_to_duration.update({ element.fullName : element.duration.quarterLength}) # 把休止符對應的duration的記錄下來
        # 如果是 Chord 型別，取音調的序號,存int型別比較容易處理
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
            notes_without_restes.append('.'.join(str(n) for n in element.normalOrder))

    


Parsing ./midi_songs\beethoven_opus10_3_format0.mid
Parsing ./midi_songs\beethoven_opus22_1_format0.mid


### 準備神經網絡使用的輸入輸出

In [3]:
## """ 準備神經網絡使用的輸入輸出 """

# 獲取音符種類名稱的數量包含休止符
n_vocab = len(set(notes))
# 獲得排序後的音符種類名稱，含休止符
pitch_and_rest = sorted(set(item for item in notes))
# 獲得排序後的音符種類名稱，不含休止符
pitchnames = sorted(set(item for item in notes_without_restes))
# 獲得排序後的休止符種類名稱
restnames = sorted(set(item for item in rests))
# 創建一個字典，把每個音符轉換分配一個對應的數字號碼 ex:(C4 > 25)，利於訓練
note_to_int = dict((note, number) for number, note in enumerate(pitch_and_rest))

"""
# 創建一個字典，把每個音符轉換分配一個對應的數字號碼 ex:(C4 > 25)，利於訓練
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
# 休止符放到字典的後面，也分配一個對應的數字號碼 
note_to_int.update((rest, number + len(pitchnames) ) for number, rest in enumerate(restnames))
"""

print("\n===== 項目解釋 ======\n")
print("notes: 是一個(list)，當中以(字串)儲存所有樂譜的音符")
print(f"樂譜裡的音符包含休止符總共有: %d 個 "  % len(notes))
print(f"樂譜裡的音符不含休止符總共有: %d 個 "  % len(pitchnames))
print(f"樂譜裡[音符種類]共有: %d 個"          %  n_vocab)
print(f"樂譜裡[休止符種類]共有: %d 個"          %  len(set(rests)))
print(f"音符種類名稱分別是: %s"               %  pitchnames)
print(f"音符種類與對應的號碼: {note_to_int} "  )



# 訓練輸入序列的長度(輸入音符的個數) 
sequence_length = 100 

network_input = [] #創建輸入序列
network_output = [] #創建輸出序列

# =====使用notes裡的音符創建輸入序列和相應的輸出=====
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]  # [0 ~ length-1 項音符], [1 ~ length 項音符], [1 ~ length+1項音符]....
    sequence_out = notes[i + sequence_length]   # 第length項音符, 第length + 1項音符,第length + 2項音符...
    
    network_input.append([note_to_int[char] for char in sequence_in]) # 加入, 把sequence_in裡的音符翻譯成的號碼
    network_output.append(note_to_int[sequence_out])    # 把sequence_out裡的音符翻譯成的號碼

print("\n===================\n")
print(f"notes: \t共有: {len(notes)} 項字串")
print(f"notes 中的每 {sequence_length} 項音符轉換成一組訓練資料 ")
print(f"network_input:  共有: {len(network_input)} 組list，每組裡有: {len(network_input[0])} 項數字")
print(f"network_output: 共有: {len(network_output)} 項數字，每項都是input的再後面一項的音符數字")
print("\n===================\n")
print("notes 第 sequence_length -10 往後10項 音符名稱分別是:\t",notes[sequence_length-10:sequence_length])
print("notes 第 sequence_length -10 往後10項 音符對應的數字是:\t", [note_to_int[char] for char in notes[sequence_length-10:sequence_length]])
print("")
print(f"network_input 第0組 的 最後10 項號碼:\t",network_input[0][sequence_length-10:sequence_length])
print(f"network_input 第1組 的 最後10 項號碼:\t",network_input[1][sequence_length-10:sequence_length])
print(f"network_input 第2組 的 最後10 項號碼:\t",network_input[2][sequence_length-10:sequence_length])
print("network_output 的第 0~2 項號碼:",network_output[0:3])






n_patterns = len(network_input)
# =====將輸入重塑為與 LSTM 層兼容的格式=====
normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# 正規化輸入
normalized_input = normalized_input / float(n_vocab)
#輸出bool矩陣，以n_vocab維度表示一個數字，用以配合categorical_crossentropy 算法
# to_categorical解釋: https://blog.csdn.net/moyu123456789/article/details/83444140
network_output = to_categorical(network_output, n_vocab)

print("\n===== 資料重塑後 =========\n")
print("normalized_input.shape:",normalized_input.shape)
print("network_output.shape:",network_output.shape)




===== 項目解釋 ======

notes: 是一個(list)，當中以(字串)儲存所有樂譜的音符
樂譜裡的音符包含休止符總共有: 6440 個 
樂譜裡的音符不含休止符總共有: 171 個 
樂譜裡[音符種類]共有: 191 個
樂譜裡[休止符種類]共有: 20 個
音符種類名稱分別是: ['0', '0.2', '0.2.5.8', '0.3', '0.3.5.8', '0.3.6', '0.3.6.8', '0.3.7', '0.4', '0.4.7', '0.5', '0.6', '1.2', '1.2.4.7.10', '1.3', '1.4.7', '1.4.7.10', '1.4.7.9', '1.5', '1.5.8', '1.7', '10', '10.0', '10.0.4', '10.1', '10.1.4', '10.1.5', '10.11', '10.2', '10.2.4', '10.2.5', '10.3', '11', '11.0', '11.2', '11.2.5', '11.2.5.7', '11.3', '2', '2.3', '2.4', '2.5', '2.5.7', '2.5.8', '2.5.8.10', '2.5.8.11', '2.5.9', '2.6', '2.6.9', '2.7', '2.8', '3', '3.4.7.10', '3.5', '3.5.9', '3.6', '3.6.10', '3.6.9.11', '3.7', '3.7.10', '3.8', '3.9', '4', '4.10', '4.5', '4.6.7.10', '4.7', '4.7.10', '4.7.10.0', '4.8', '4.9', '5', '5.10', '5.11', '5.6', '5.7', '5.7.8', '5.8', '5.8.0', '5.8.10', '5.8.11', '5.8.11.0', '5.9', '5.9.0', '6', '6.8.0', '6.9', '6.9.0', '6.9.0.2', '7', '7.0', '7.10', '7.10.0', '7.10.1.3', '7.10.2', '7.11', '7.11.2', '7.9', '8', '8.0', '8.0

### 創建神經網絡的結構 
### LSTM

In [4]:
##""" 創建神經網絡的結構 """
#LSTM

# https://huhuhang.com/post/machine-learning/lstm-return-sequences-state
model = Sequential()
model.add(LSTM(
        512,
        input_shape=(normalized_input.shape[1], normalized_input.shape[2]),
        recurrent_dropout=0.1,
        return_sequences=True
    ))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.1,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.1))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.1))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                        

### 訓練神經網絡

In [5]:
## """ 訓練神經網絡 """
# callbacks = ModelCheckpoint('model{epoch:03d}.weights.h5', save_weights_only=True)
# model.fit(network_input, network_output, epochs=100, batch_size=128, callbacks=callbacks)

history = model.fit(normalized_input, network_output, epochs=1, batch_size=128, callbacks=None)
# epoch == 100


50/50 [==============================] - 70s 1s/step - loss: 5.0684


根據選定的音符起始點，從神經網絡預測下一個音符並生成樂譜

In [ ]:
##""" 根據選定的音符起始點，從神經網絡預測下一個音符並生成樂譜 """

# 從 network_input 中選擇一個組隨機序列作為預測的起點
start = numpy.random.randint(0, len(network_input)-1)
# 作為預測 起始點的一串長度為sequence_length的音符
pattern = network_input[start]

# 把數字還原回音符的字典
int_to_note = dict((number, note) for number, note in enumerate(pitch_and_rest)) 


prediction_output = []
print("生成的音符:")

# 隨機生成n個音符
for note_index in range(100):#更改range()改變音符生成的數量
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab) #正規化

    #預測每一個音符的概率
    prediction = model.predict(prediction_input, verbose=0) 


    #挑選prediction_output裡最大的值
    index = numpy.argmax(prediction)

    #提取對應的音符
    result = int_to_note[index]
    print(result)
    prediction_output.append(result)

    # 將預測的一個音符放入預測窗口sequence_length，放掉原本窗口內最左邊的一個音符(窗口往右移動)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]


<class 'list'>
5
7
16th Rest
9
16th Rest
10
16th Rest
0
16th Rest
2
16th Rest
3
2
16th Rest
0
10
16th Rest
9
7
16th Rest
5
16th Rest
3
16th Rest
2
16th Rest
0
16th Rest
10
Dotted Eighth Rest
9.0.3.5
G#4
A4
F4
10.2
Dotted Eighth Rest
9.0.3.5
G#5
A5
F5
10.2
Double Dotted Quarter Rest
9.0.3.5
Quarter Rest
10.2.5
Dotted Half Rest
Half Rest
D4
E-4
F4
B-2
16th Rest
B-3
D4
B-3
7.9
E-4
F4
7.0
16th Rest
A4
B-4
D4
C#4
F4
D4
B-3
16th Rest
7.9
E-4
F4
7.0
16th Rest
A4
B-4
D4
16th Rest
C#4
16th Rest
A4
B-4
D4
16th Rest
A3
16th Rest
C5
D5
B-3
16th Rest
G3
16th Rest
E5
4.10
F3
5.9
Eighth Rest
Dotted Quarter Rest
3.5
D5
E-5


將預測的輸出轉換為音符，並從音符中創建一個MIDI文件 

In [ ]:
## """ 將預測的輸出轉換為音符，並從音符中創建一個MIDI文件 """

offset = 0
output_notes = []

# 根據模型生成的值創建音符和和弦對象
for element in prediction_output:
    # 模式是和弦
    if ('.' in element) or element.isdigit():
        notes_in_chord = element.split('.')
        notes = []
        for current_note in notes_in_chord:
            
            try:
                new_note = note.Note(int(current_note))
#                 print(new_note)

            except ValueError:
                   pass 
                
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)

    # 模式是一個音符
    elif( element in pitchnames):
        new_note = note.Note(element)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # 模式是一個休止符
    elif( element in restnames):
        # restname_list = element.split(' ')
        # restname = " ".join(restname_list[0:-1])
        new_note = note.Rest(rest_to_duration[element])
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # 音符之間的間距
    offset += 0.5
    
# 創建樂譜
midi_stream = stream.Stream(output_notes)

# 創建 MIDI 文件
midi_stream.write('midi', fp='LAB3B_LSTM_music_(rest).mid')
